<a href="https://colab.research.google.com/github/kingmask420/newjoke/blob/main/Fine_Tuning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.1 MB/s 
     |████████████████████████████████| 115 kB 65.6 MB/s 
     |████████████████████████████████| 120 kB 61.2 MB/s 
     |████████████████████████████████| 212 kB 66.5 MB/s 
     |████████████████████████████████| 127 kB 64.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import datasets

snli = datasets.load_dataset('snli', split='train')

snli

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 550152
})

In [ ]:
mnli = datasets.load_dataset('glue', 'mnli', split='train')
mnli

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 392702
})

In [ ]:
mnli = mnli.remove_columns(['idx'])
mnli

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 392702
})

In [ ]:
snli.cast(mnli.features)

Casting the dataset:   0%|          | 0/56 [00:00<?, ?ba/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 550152
})

In [ ]:
dataset = datasets.concatenate_datasets([snli, mnli])

In [ ]:
dataset

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 942854
})

In [ ]:
dataset = dataset.filter(
    lambda x : False if x['label'] != 0 else True
)
dataset

  0%|          | 0/943 [00:00<?, ?ba/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 314315
})

In [ ]:
!pip install -U sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 4.7 MB 65.6 MB/s 
     |████████████████████████████████| 1.3 MB 55.5 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=345dcbb86b628d013777c00ca95dfb67128b9647e3fafd4743cb07dd73b48425
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import InputExample
from tqdm.auto import tqdm

samples = []
for row in tqdm(dataset):
  samples.append(InputExample(
      texts=[row['premise'], row['hypothesis']]
  ))

  0%|          | 0/314315 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import datasets

batch_size = 32

loader = datasets.NoDuplicatesDataLoader(
    samples, batch_size = batch_size
)

In [ ]:
from sentence_transformers import models, SentenceTransformer

bert = models.Transformer('bert-base-uncased')
pooling = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens = True
)

model = SentenceTransformer(modules=[bert, pooling])
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
0.1 * len(loader)

982.2

In [ ]:
epochs = 1
warmup = int(0.02 * len(loader))

model.fit(
    train_objectives=[(loader, loss)],
    epochs = epochs,
    warmup_steps=warmup,
    output_path="bert_model",
    show_progress_bar=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9822 [00:00<?, ?it/s]

RuntimeError: ignored